In [9]:
import os
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0

from sklearn.model_selection import train_test_split


from PIL import Image

In [28]:
image_dir = '/Users/mjs/Desktop/Dev/TACO_MJS/data/all_images'

train_df = pd.read_csv('./data/splits/train_recycle_partially_balanced_2023-04-11.csv')
val_df = pd.read_csv('./data/splits/val_recycle_2023-04-11.csv')


In [29]:
img_size = 200
batch_size = 12
epochs = 10
learning_rate = 0.0001

labels = list(train_df.columns)[1:]



# Define data generators for train and validation sets
datagen_train = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.005,
    zoom_range=[0.9, 1.4],
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=(.8,1.2),
    fill_mode='nearest',
)

datagen_val = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    directory='/Users/mjs/Desktop/Dev/TACO/data/all_images',
    x_col='file_name',
    y_col=labels,
    class_mode='raw',
    target_size=(img_size, img_size), 
    batch_size=batch_size,
    shuffle=True,
)

val_generator = datagen_val.flow_from_dataframe(
    dataframe=val_df,
    directory='/Users/mjs/Desktop/Dev/TACO/data/all_images',
    x_col='file_name',
    y_col=labels,
    class_mode='raw',
    target_size=(img_size, img_size), 
    batch_size=batch_size,
    shuffle=False,
)

Found 3625 validated image filenames.
Found 199 validated image filenames.


In [30]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))


for layer in base_model.layers:
    layer.trainable = True

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = Dense(len(labels), activation='sigmoid')(x)


model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, monitor="val_loss", restore_best_weights=True)


optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, 
              loss='binary_crossentropy', 
              metrics=['accuracy'],
              )


In [ ]:



#checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)


history = model.fit(train_generator,
                    validation_data=val_generator,
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping_cb]
                   )


history_dict = history.history
'''
for key, val in history_dict.items():
    if isinstance(val, np.ndarray):
        history_dict[key] = val.tolist()

with open('history.json', 'w') as outfile:
    json.dump(history_dict, outfile)'''



Epoch 1/10
303/303 [==============================] - 798s 3s/step - loss: 0.4456 - accuracy: 0.5992 - val_loss: 0.6429 - val_accuracy: 0.2513
Epoch 2/10
271/303 [=========================>....] - ETA: 1:20 - loss: 0.2996 - accuracy: 0.6442

In [129]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_generator)
print(f'Validation loss: {loss:.3f}')
print(f'Validation accuracy: {accuracy:.3f}')

28/28 [==============================] - 25s 894ms/step - loss: 0.1221 - accuracy: 0.0060
Validation loss: 0.122
Validation accuracy: 0.006
